In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/all_models/scikitlearn/default/1/Integrity_Impact_best_catboost_model.pkl
/kaggle/input/all_models/scikitlearn/default/1/Impact_Score_xgb_regressor.pkl
/kaggle/input/all_models/scikitlearn/default/1/accessVector_best_catboost_model.pkl
/kaggle/input/all_models/scikitlearn/default/1/Availability_Impact_best_catboost_model.pkl
/kaggle/input/all_models/scikitlearn/default/1/Availability_Impact_label_encoder.pkl
/kaggle/input/all_models/scikitlearn/default/1/Exploitability_Score_xgb_regressor.pkl
/kaggle/input/all_models/scikitlearn/default/1/Access_Complexity_label_encoder.pkl
/kaggle/input/all_models/scikitlearn/default/1/Integrity_Impactt_label_encoder.pkl
/kaggle/input/all_models/scikitlearn/default/1/Access_Complexity_best_catboost_model.pkl
/kaggle/input/all_models/scikitlearn/default/1/Confidentiality_Impact_best_catboost_model.pkl
/kaggle/input/all_models/scikitlearn/default/1/Confidentiality_Impact_label_encoder.pkl
/kaggle/input/all_models/scikitlearn/default/1/Acce

In [1]:
! pip install xgboost
!pip install lightgbm catboost torch torchvision torchaudio
!pip install sentence-transformers


In [10]:
import numpy as np
import pickle
import warnings
from sentence_transformers import SentenceTransformer

# Suppress all warnings
warnings.filterwarnings("ignore")

# Load the embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def predict_metrics(description):
    # Function to convert a single description to embedding
    embedding = model.encode(description, convert_to_tensor=True).cpu().numpy().reshape(1, -1)

    # Define columns to exclude for input features
    columns_to_exclude = [
        'Impact_Score', 'CVE_ID', 'ASSIGNER', 'Description', 
        'Published_Date', 'Last_Modified_Date', 
        'Access_Vector', 'Access_Complexity', 
        'Configurations', 'Reference_Data',
        'Year', 'Base_Score', 'Exploitability_Score', 
        'Confidentiality_Impact', 'Integrity_Impact', 
        'Availability_Impact'
    ]

    # Define paths for models stored on Kaggle
    base_path = r'C:\Users\pradeep dubey\Desktop\NLP_Project\MODELS2'
    # Load regression and classification models
    regression_models = {
        "Impact_Score": base_path + "\Impact_Score_xgb_regressor.pkl",
        "Base_Score": base_path + "\BASE_SCORE_xgb_regressor.pkl",
        "Exploitability_Score": base_path + "\Exploitability_Score_xgb_regressor.pkl"
    }

    classification_models = {
        "Access_Complexity": (base_path + "\\Access_Complexity_best_catboost_model.pkl", base_path + "\\Access_Complexity_label_encoder.pkl"),
        "Access_Vector": (base_path + "\\accessVector_best_catboost_model.pkl", base_path + "\\Access_Vector_label_encoder.pkl"),
        "Availability_Impact": (base_path + "\\Availability_Impact_best_catboost_model.pkl", base_path + "\\Availability_Impact_label_encoder.pkl"),
        "Confidentiality_Impact": (base_path + "\\Confidentiality_Impact_best_xgboost_model.pkl", base_path + "\\Confidentiality_Impact_label_encoder.pkl"),
        "Integrity_Impact": (base_path + "\\Integrity_Impact_best_xgboost_model.pkl", base_path + "\\Integrity_Impact_label_encoder.pkl"),
    }

    # Initialize dictionary to hold predicted values
    predicted_outputs = {}

    # Predict regression targets
    for target, model_file in regression_models.items():
        with open(model_file, 'rb') as file:
            loaded_model = pickle.load(file)
        
        predicted_output = loaded_model.predict(embedding).flatten()  # Use flatten for regression output
        predicted_outputs[target] = predicted_output[0]  # Store predicted value

    # Predict classification targets with label decoding
    for target, (model_file, encoder_file) in classification_models.items():
        with open(encoder_file, 'rb') as f:
            label_encoder = pickle.load(f)

        with open(model_file, 'rb') as f:
            best_model = pickle.load(f)

        predicted_label_encoded = best_model.predict(embedding).flatten()  # Ensure 1D
        predicted_label = label_encoder.inverse_transform(predicted_label_encoded.astype(int))
        predicted_outputs[target] = predicted_label[0]  # Store predicted label

    # Return predicted values for all outputs
    return predicted_outputs


In [11]:
# Example usage
new_description = "The Certificate Trust Policy component in Apple Mac OS X before 10.6.8 does not perform CRL checking for Extended Validation (EV) certificates that lack OCSP URLs, which might allow man-in-the-middle attackers to spoof an SSL server via a revoked certificate."
predicted_metrics = predict_metrics(new_description)

# Display only predicted metrics
print("Predicted Outputs:")
for key, value in predicted_metrics.items():
    print(f"{key}: {value}")

Predicted Outputs:
Impact_Score: 3.537036895751953
Base_Score: 5.620712757110596
Exploitability_Score: 6.123106956481934
Access_Complexity: MEDIUM
Access_Vector: NETWORK
Availability_Impact: NONE
Confidentiality_Impact: NONE
Integrity_Impact: NONE
